# AGENT #

An agent, as defined in 2.1 is anything that can perceive its <b>environment</b> through sensors, and act upon that environment through actuators based on its <b>agent program</b>. This can be a dog, robot, or even you. As long as you can perceive the environment and act on it, you are an agent. This notebook will explain how to implement a simple agent, create an environment, and create a program that helps the agent act on the environment based on its percepts.

Before moving on, review the </b>Agent</b> and </b>Environment</b> classes in <b>[agents.py](https://github.com/aimacode/aima-python/blob/master/agents.py)</b>.

Let's begin by importing all the functions from the agents.py module and creating our first agent - a blind dog.

In [20]:
from agents import *

class BlindDog(Agent):
    def eat(self, thing):
        print("Dog: Ate food at {}.".format(self.location))
            
    def drink(self, thing):
        print("Dog: Drank water at {}.".format( self.location))

dog = BlindDog()

What we have just done is create a dog who can only feel what's in his location (since he's blind), and can eat or drink. Let's see if he's alive...

In [2]:
print(dog.alive)

True


![Cool dog](https://gifgun.files.wordpress.com/2015/07/wpid-wp-1435860392895.gif)
This is our dog. How cool is he? Well, he's hungry and needs to go search for food. For him to do this, we need to give him a program. But before that, let's create a park for our dog to play in.

# ENVIRONMENT #

A park is an example of an environment because our dog can perceive and act upon it. The <b>Environment</b> class in agents.py is an abstract class, so we will have to create our own subclass from it before we can use it. The abstract class must contain the following methods:

<li><b>percept(self, agent)</b> - returns what the agent perceives</li>
<li><b>execute_action(self, agent, action)</b> - changes the state of the environment based on what the agent does.</li>

In [32]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park(Environment):
    def percept(self, agent):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        print(things)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog pick eat the first item
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will or it with when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


## Wumpus Environment

In [29]:
from ipythonblocks import BlockGrid
from agents import *

color = {"Breeze": (225, 225, 225),
        "Pit": (0,0,0),
        "Gold": (253, 208, 23),
        "Glitter": (253, 208, 23),
        "Wumpus": (43, 27, 23),
        "Stench": (128, 128, 128),
        "Explorer": (0, 0, 255),
        "Wall": (44, 53, 57)
        }

def program(percepts):
    '''Returns an action based on it's percepts'''
    print(percepts)
    return input()

w = WumpusEnvironment(program, 7, 7)         
grid = BlockGrid(w.width, w.height, fill=(123, 234, 123))

def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                grid[y, x] = color[world[x][y][-1].__class__.__name__]

def step():
    global grid, w
    draw_grid(w.get_world())
    grid.show()
    w.step()

In [2]:
step()

,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


[[<Bump>], [None], [<Bump>], [None], [None]]



# PROGRAM #
Now that we have a <b>Park</b> Class, we need to implement a <b>program</b> module for our dog. A program controls how the dog acts upon it's environment. Our program will be very simple, and is shown in the table below.
<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>move up</td>
   </tr>
        
</table>


In [33]:
class BlindDog(Agent):
    location = 1
    
    def movedown(self):
        self.location += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            print("Dog: Ate food at {}.".format(self.location))
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            print("Dog: Drank water at {}.".format(self.location))
            return True
        return False
        
def program(percepts):
    '''Returns an action based on it's percepts'''
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    return 'move down'
       
                

In [34]:
park = Park()
dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, 0)
park.add_thing(dogfood, 5)
park.add_thing(water, 7)

park.run(10)

[<BlindDog>]
[<BlindDog>]
[<BlindDog>]
[<BlindDog>]
[<BlindDog>]
[<BlindDog>, <Food>]
Dog: Ate food at 5.
[<BlindDog>]
[<BlindDog>]
[<BlindDog>, <Water>]
Dog: Drank water at 7.


That's how easy it is to implement an agent, its program, and environment. But that was a very simple case. What if our environment was 2-Dimentional instead of 1? And what if we had multiple agents?

To make our Park 2D, we will need to make it a subclass of <b>XYEnvironment</b> instead of Environment. Also, let's add a person to play fetch with the dog.

In [28]:
from agents import *

class MAgent(Agent):
    
    def __init__(self,program):
        super().__init__(program)
        self.things = []
    
    def symbol(self):
        return "a "   
    
    def add_perecpted_things(self,things):
        for thing in things:
            self.things.append(thing)
    
    def thing_at(self,location):
        for thing in self.things:
            if thing.location == location:
                return thing.symbol()
        return "? "

    
    def show_state(self):
        print("  0 1 2 3 4 5 ")
        for y in range(6):
            print(y,end=' ')
            for x in range(6):
                print(self.thing_at((x,y)),end='')
            print()
        
class Nothing(Thing):
    def __init__(self,location):
        self.location = location
    def symbol(self):
        return "- "

class Outside(Thing):
    def __init__(self,location):
        self.location = location
    def symbol(self):
        return "o "

class Cosa(Thing):
    def symbol(self):
        return "c "

class MEnv(Environment):
    
    def symbol(self,thing):
        if isinstance(thing,Outside):
            return "o "
        elif isinstance(thing,Nothing):
            return "- "
        else:
            return thing.symbol()
        
    def thing_at(self, location):
        if location[0] < 0 or location[1] < 0:
            return Outside(location)
        elif self.list_things_at(location):
            return self.list_things_at(location)[0]
        else:
            return Nothing(location)
        
    def percept(self, agent):
        location = agent.location
        
        here = self.thing_at(location)                       # Agent's Location
        left = self.thing_at((location[0] - 1,location[1]))  # Left
        down = self.thing_at((location[0],location[1] + 1))  # Down
        right = self.thing_at((location[0] + 1,location[1])) # Right
        up = self.thing_at((location[0],location[1] - 1))    # Up        
        print("Percept: ({},({},{},{},{},{}))".format(location,self.symbol(here),self.symbol(left),self.symbol(down),self.symbol(up),self.symbol(right)))
        return [here,left,down,up,right]
    
    def execute_action(self,agent, action):
        pass
        
    def draw_grid(self):
        print("  0 1 2 3 4 5 ")
        for y in range(6):
            print(y,end=' ')
            for x in range(6):
                print(self.symbol(self.thing_at((x,y))),end='')
            print()
                    
e = MEnv()
c = Cosa()
e.add_thing(c,(2,2))
e.add_thing(Cosa(),(4,4))

def program(percepts):
    pass

agent = MAgent(program)
TraceAgent(agent)
e.add_thing(agent,(2,2))
e.draw_grid()
print()
agent.add_perecpted_things(e.percept(agent))
agent.show_state()
e.run(3)

  0 1 2 3 4 5 
0 - - - - - - 
1 - - - - - - 
2 - - c - - - 
3 - - - - - - 
4 - - - - c - 
5 - - - - - - 

Percept: ((2, 2),(c ,- ,- ,- ,- ))
  0 1 2 3 4 5 
0 ? ? ? ? ? ? 
1 ? ? - ? ? ? 
2 ? - c - ? ? 
3 ? ? - ? ? ? 
4 ? ? ? ? ? ? 
5 ? ? ? ? ? ? 
Percept: ((2, 2),(c ,- ,- ,- ,- ))
<MAgent> perceives [<Cosa>, <Nothing>, <Nothing>, <Nothing>, <Nothing>] and does None
Percept: ((2, 2),(c ,- ,- ,- ,- ))
<MAgent> perceives [<Cosa>, <Nothing>, <Nothing>, <Nothing>, <Nothing>] and does None
Percept: ((2, 2),(c ,- ,- ,- ,- ))
<MAgent> perceives [<Cosa>, <Nothing>, <Nothing>, <Nothing>, <Nothing>] and does None


In [180]:
cha = np.chararray((6,6))
cha[2,2] = "z"

k = []

(1,2)[0]

k = 1
k = "3"
k

'3'

In [26]:
i = 0
i+= 1
i

1